In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set seed
SEED = 10
np.random.seed(SEED)

# Load dữ liệu
df = pd.read_csv('C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv')
features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
X = df[features].astype(float)
y = df['Gas Used'].astype(float)

# Chia tập train/validation (không bắt buộc cho K-Fold nhưng vẫn làm cho đồng nhất)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Scale dữ liệu
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = pd.DataFrame(scaler_X.fit_transform(X), columns=X.columns)
y_scaled = pd.Series(scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten(), name='Gas Used')

# K-Fold
kf = KFold(n_splits=10, shuffle=True, random_state=SEED)

# Đánh giá
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled), start=1):
    X_tr = X_train_scaled.iloc[train_idx]
    y_tr = y_train_scaled[train_idx]
    X_val = X_train_scaled.iloc[val_idx]
    y_val = y_train_scaled[val_idx]

    model = LGBMRegressor()
             # <- mở nếu LightGBM của bạn hỗ trợ GP )

    # Early stopping dùng callback (tương thích nhiều version LightGBM)
    callbacks = [
        lgb.early_stopping(stopping_rounds=200, verbose=False),
        lgb.log_evaluation(period=0)  # tắt log mỗi vòng
    ]

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=callbacks
    )

    # Dự đoán dùng best_iteration_ (nếu có)
    y_pred = model.predict(X_val, num_iteration=getattr(model, "best_iteration_", None))

    # Metrics (trên thang đã scale)
    mse  = mean_squared_error(y_val, y_pred)
    mae  = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_val, y_pred)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

  
    print(f"Fold {fold} - MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

# Tổng kết sau tất cả folds
print("\n=== Kết quả trung bình (scaled) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")

Dropped constant/NaN columns: ['Transaction_Hash_len', 'signature_len', 'From_len', 'To_len', 'sender_len', 'paymaster_len']
Fold 1 - MSE: 173496373069.727020 | MAE: 277753.761795 | RMSE: 416528.958261 | R²: -1.039461
Fold 2 - MSE: 173496373069.727020 | MAE: 277753.761795 | RMSE: 416528.958261 | R²: -1.039461
Fold 3 - MSE: 173496373069.727020 | MAE: 277753.761795 | RMSE: 416528.958261 | R²: -1.039461


KeyboardInterrupt: 

In [9]:
epochs = 20   # bạn đổi thành 30 nếu muốn
k = 10
cats = np.zeros((epochs, 4))  # lưu [RMSE, MSE, MAE, R2] cho mỗi epoch

for ep in range(epochs):
    kf = KFold(n_splits=k, shuffle=True, random_state=SEED + ep)

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for train_idx, val_idx in kf.split(X_train_scaled):
        X_tr, X_va = X_train_scaled.iloc[train_idx], X_train_scaled.iloc[val_idx]
        y_tr, y_va = y_train_scaled[train_idx], y_train_scaled[val_idx]

        model = LGBMRegressor()
            
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_va)

        mse  = mean_squared_error(y_va, y_pred)
        rmse = np.sqrt(mse)
        mae  = mean_absolute_error(y_va, y_pred)
        r2   = r2_score(y_va, y_pred)

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    # lưu mean metrics của epoch này (qua 10 folds)
    cats[ep, :] = [
        np.mean(rmse_list),
        np.mean(mse_list),
        np.mean(mae_list),
        np.mean(r2_list)
    ]

# mean cuối cùng qua tất cả epochs
final_mean = np.mean(cats, axis=0)

print(final_mean)  # 1 dòng mean cuối cùng
print(cats)        # bảng (epochs × 4)

NameError: name 'X_train_scaled' is not defined